In [1]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests

In [2]:
url_exp = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/gov_10a_exp/1.0/*.*.*.*.*.*?c[freq]=A&c[unit]=PC_GDP&c[sector]=S13&c[cofog99]=GF01,GF07,GF09,GF1002&c[na_item]=TE&c[geo]=EU27_2020,DE,ES,IT,AT,SE&c[TIME_PERIOD]=2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990&compress=true&format=csvdata&formatVersion=2.0&lang=en&labels=both"
url_taxag = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/gov_10a_taxag/1.0/*.*.*.*.*?c[freq]=A&c[unit]=PC_GDP&c[sector]=S13&c[na_item]=D51C1,D51C2,D51C3,D51D,D51E,D59,D61,D91&c[geo]=EU27_2020,DE,ES,IT,AT,SE&c[TIME_PERIOD]=2024,2023,2022,2021,2020,2019,2018,2017,2016,2015&compress=true&format=csvdata&formatVersion=2.0&lang=en&labels=name"

In [3]:
df_raw_exp = requests.get(url_exp, params=None)
df_raw_exp.raise_for_status()
df_raw_exp = pd.read_csv(
    url_exp,
    sep=',',
    compression='gzip',
    dtype=str,        # o tu mapeo de dtypes
    na_values=[':']   # si quieres interpretar ':' como NaN
)

df_raw_taxag = requests.get(url_taxag, params=None)
df_raw_taxag.raise_for_status()
df_raw_taxag = pd.read_csv(
    url_taxag,
    sep=',',
    compression='gzip',
    dtype=str,        # o tu mapeo de dtypes
    na_values=[':']   # si quieres interpretar ':' como NaN
)

In [4]:
df_raw_exp.head()

,STRUCTURE,STRUCTURE_ID,freq: Time frequency,unit: Unit of measure,sector: Sector,cofog99: Classification of the functions of government (COFOG 1999),na_item: National accounts indicator (ESA 2010),geo: Geopolitical entity (reporting),TIME_PERIOD: Time,OBS_VALUE: Observation value,OBS_FLAG: Observation status (Flag) V2 structure,CONF_STATUS: Confidentiality status (flag)
0,dataflow,ESTAT:GOV_10A_EXP(1.0),A: Annual,PC_GDP: Percentage of gross domestic product (...,S13: General government,GF01: General public services,TE: Total general government expenditure,AT: Austria,1995,9.3,NaN,NaN
1,dataflow,ESTAT:GOV_10A_EXP(1.0),A: Annual,PC_GDP: Percentage of gross domestic product (...,S13: General government,GF01: General public services,TE: Total general government expenditure,AT: Austria,1996,9.5,NaN,NaN
2,dataflow,ESTAT:GOV_10A_EXP(1.0),A: Annual,PC_GDP: Percentage of gross domestic product (...,S13: General government,GF01: General public services,TE: Total general government expenditure,AT: Austria,1997,8.7,NaN,NaN
3,dataflow,ESTAT:GOV_10A_EXP(1.0),A: Annual,PC_GDP: Percentage of gross domestic product (...,S13: General government,GF01: General public services,TE: Total general government expenditure,AT: Austria,1998,8.6,NaN,NaN
4,dataflow,ESTAT:GOV_10A_EXP(1.0),A: Annual,PC_GDP: Percentage of gross domestic product (...,S13: General government,GF01: General public services,TE: Total general government expenditure,AT: Austria,1999,8.5,NaN,NaN


In [5]:
df_raw_exp.dtypes

STRUCTURE                                                              object
STRUCTURE_ID                                                           object
freq: Time frequency                                                   object
unit: Unit of measure                                                  object
sector: Sector                                                         object
cofog99: Classification of the functions of government (COFOG 1999)    object
na_item: National accounts indicator (ESA 2010)                        object
geo: Geopolitical entity (reporting)                                   object
TIME_PERIOD: Time                                                      object
OBS_VALUE: Observation value                                           object
OBS_FLAG: Observation status (Flag) V2 structure                       object
CONF_STATUS: Confidentiality status (flag)                             object
dtype: object

In [6]:
df_raw_taxag.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,sector,Sector,na_item,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:GOV_10A_TAXAG(1.0),Main national accounts tax aggregates,A,Annual,PC_GDP,Percentage of gross domestic product (GDP),S13,General government,D51C1,...,AT,Austria,2015,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:GOV_10A_TAXAG(1.0),Main national accounts tax aggregates,A,Annual,PC_GDP,Percentage of gross domestic product (GDP),S13,General government,D51C1,...,AT,Austria,2016,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:GOV_10A_TAXAG(1.0),Main national accounts tax aggregates,A,Annual,PC_GDP,Percentage of gross domestic product (GDP),S13,General government,D51C1,...,AT,Austria,2017,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:GOV_10A_TAXAG(1.0),Main national accounts tax aggregates,A,Annual,PC_GDP,Percentage of gross domestic product (GDP),S13,General government,D51C1,...,AT,Austria,2018,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:GOV_10A_TAXAG(1.0),Main national accounts tax aggregates,A,Annual,PC_GDP,Percentage of gross domestic product (GDP),S13,General government,D51C1,...,AT,Austria,2019,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [7]:
df_raw_taxag.dtypes

STRUCTURE                                 object
STRUCTURE_ID                              object
STRUCTURE_NAME                            object
freq                                      object
Time frequency                            object
unit                                      object
Unit of measure                           object
sector                                    object
Sector                                    object
na_item                                   object
National accounts indicator (ESA 2010)    object
geo                                       object
Geopolitical entity (reporting)           object
TIME_PERIOD                               object
Time                                      object
OBS_VALUE                                 object
Observation value                         object
OBS_FLAG                                  object
Observation status (Flag) V2 structure    object
CONF_STATUS                               object
Confidentiality stat

In [8]:
# 1) Limpieza de nombres de columna a snake_case
def clean_cols(df):
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r'[\s:/().-]+', '_', regex=True)
          .str.replace(r'_+', '_', regex=True)
          .str.strip('_')
    )
    return df

# 2) Aplica limpieza sobre df_raw_exp
df_exp = clean_cols(df_raw_exp)

# 3) Renombra sólo las columnas clave
df_exp = df_exp.rename(columns={
    'cofog99_classification_of_the_functions_of_government_cofog_1999': 'cofog',
    'na_item_national_accounts_indicator_esa_2010':                  'na_item',
    'geo_geopolitical_entity_reporting':                            'geo',
    'time_period_time':                                             'time_period',
    'obs_value_observation_value':                                  'obs_value'
})

# 4) Selecciona sólo esas columnas
df_exp = df_exp[['geo', 'time_period', 'cofog', 'na_item', 'obs_value']]

# 5) Extrae el código ISO ante los dos puntos
df_exp['geo'] = df_exp['geo'].str.split(':').str[0].str.strip()

# 6) Muestra la cabecera y los valores únicos de geo
print(df_exp.head(10))
print("Geos únicos en df_exp:", sorted(df_exp['geo'].unique()))

  geo time_period                          cofog  \
0  AT        1995  GF01: General public services   
1  AT        1996  GF01: General public services   
2  AT        1997  GF01: General public services   
3  AT        1998  GF01: General public services   
4  AT        1999  GF01: General public services   
5  AT        2000  GF01: General public services   
6  AT        2001  GF01: General public services   
7  AT        2002  GF01: General public services   
8  AT        2003  GF01: General public services   
9  AT        2004  GF01: General public services   

                                    na_item obs_value  
0  TE: Total general government expenditure       9.3  
1  TE: Total general government expenditure       9.5  
2  TE: Total general government expenditure       8.7  
3  TE: Total general government expenditure       8.6  
4  TE: Total general government expenditure       8.5  
5  TE: Total general government expenditure       8.3  
6  TE: Total general government exp

In [9]:
df_exp.head()

,geo,time_period,cofog,na_item,obs_value
0,AT,1995,GF01: General public services,TE: Total general government expenditure,9.3
1,AT,1996,GF01: General public services,TE: Total general government expenditure,9.5
2,AT,1997,GF01: General public services,TE: Total general government expenditure,8.7
3,AT,1998,GF01: General public services,TE: Total general government expenditure,8.6
4,AT,1999,GF01: General public services,TE: Total general government expenditure,8.5


In [10]:
# 1) Limpieza inicial de nombres
def clean_cols(df):
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r'[\s:/().-]+', '_', regex=True)
          .str.replace(r'_+', '_', regex=True)
          .str.strip('_')
    )
    return df

df_tax = clean_cols(df_raw_taxag)

# 2) Quitar duplicados de columnas
df_tax = df_tax.loc[:, ~df_tax.columns.duplicated()]

# 3) Eliminar la columna 'geo' original
df_tax = df_tax.drop(columns=['geo'])

# 4) Renombrar la columna de entidad a 'geo'
df_tax = df_tax.rename(columns={
    'geopolitical_entity_reporting': 'geo',
    'time_period':                   'time_period',
    'structure_name':                'structure_name',
    'na_item':                       'na_item',
    'obs_value':                     'obs_value'
})

# 5) Seleccionar solo las columnas clave
df_tax = df_tax[['geo', 'time_period', 'structure_name', 'na_item', 'obs_value']]

# 6) Extraer antes de ':' en geo (por si viniera "Spain: España")
df_tax['geo'] = df_tax['geo'].str.split(':').str[0].str.strip()

# 7) Mostrar resultados
print(df_tax.head(10))
print("Geos únicos en df_tax:", sorted(df_tax['geo'].unique()))

       geo time_period                         structure_name na_item  \
0  Austria        2015  Main national accounts tax aggregates   D51C1   
1  Austria        2016  Main national accounts tax aggregates   D51C1   
2  Austria        2017  Main national accounts tax aggregates   D51C1   
3  Austria        2018  Main national accounts tax aggregates   D51C1   
4  Austria        2019  Main national accounts tax aggregates   D51C1   
5  Austria        2020  Main national accounts tax aggregates   D51C1   
6  Austria        2021  Main national accounts tax aggregates   D51C1   
7  Austria        2022  Main national accounts tax aggregates   D51C1   
8  Austria        2023  Main national accounts tax aggregates   D51C1   
9    Italy        2015  Main national accounts tax aggregates   D51C1   

  obs_value  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4       0.0  
5       0.0  
6       0.0  
7       0.0  
8       0.0  
9       0.2  
Geos únicos en df_tax: ['Austria', 'Europe

In [11]:
df_tax.sample(10)

,geo,time_period,structure_name,na_item,obs_value
373,Sweden,2021,Main national accounts tax aggregates,D91,NaN
305,Italy,2017,Main national accounts tax aggregates,D61,13.0
151,Sweden,2019,Main national accounts tax aggregates,D51D,NaN
283,Germany,2022,Main national accounts tax aggregates,D61,16.9
138,Italy,2015,Main national accounts tax aggregates,D51D,0.0
311,Italy,2023,Main national accounts tax aggregates,D61,12.7
54,Sweden,2023,Main national accounts tax aggregates,D51C2,NaN
161,Austria,2019,Main national accounts tax aggregates,D51E,0.3
120,Spain,2015,Main national accounts tax aggregates,D51D,0.0
119,Germany,2023,Main national accounts tax aggregates,D51D,NaN


In [12]:
# Diccionario para mapear nombres completos a códigos
name_to_code = {
    'Austria':                                   'AT',
    'Germany':                                   'DE',
    'Spain':                                     'ES',
    'Sweden':                                    'SE',
    'Italy':                                     'IT',
    'European Union - 27 countries (from 2020)': 'EU27_2020'
}

# 1) Aplicar el mapeo
df_tax['geo'] = df_tax['geo'].replace(name_to_code)

# 2) Mostrar cabecera y códigos únicos resultantes
print(df_tax.head(10))
print("Geos únicos en df_tax tras mapear:", sorted(df_tax['geo'].unique()))

  geo time_period                         structure_name na_item obs_value
0  AT        2015  Main national accounts tax aggregates   D51C1       0.0
1  AT        2016  Main national accounts tax aggregates   D51C1       0.0
2  AT        2017  Main national accounts tax aggregates   D51C1       0.0
3  AT        2018  Main national accounts tax aggregates   D51C1       0.0
4  AT        2019  Main national accounts tax aggregates   D51C1       0.0
5  AT        2020  Main national accounts tax aggregates   D51C1       0.0
6  AT        2021  Main national accounts tax aggregates   D51C1       0.0
7  AT        2022  Main national accounts tax aggregates   D51C1       0.0
8  AT        2023  Main national accounts tax aggregates   D51C1       0.0
9  IT        2015  Main national accounts tax aggregates   D51C1       0.2
Geos únicos en df_tax tras mapear: ['AT', 'DE', 'ES', 'EU27_2020', 'IT', 'SE']


In [13]:
# Lista definitiva de códigos
country_codes = ['ES', 'DE', 'IT', 'SE', 'EU27_2020']

# Filtrado final
df_exp = df_exp[df_exp['geo'].isin(country_codes)].copy()
df_tax = df_tax[df_tax['geo'].isin(country_codes)].copy()

# Inspección: forma y primeras filas
print("=== df_exp filtrado ===")
print("Shape:", df_exp.shape)
print(df_exp.head(5))

print("\n=== df_tax filtrado ===")
print("Shape:", df_tax.shape)
print(df_tax.head(5))

=== df_exp filtrado ===
Shape: (557, 5)
   geo time_period                          cofog  \
29  DE        1995  GF01: General public services   
30  DE        1996  GF01: General public services   
31  DE        1997  GF01: General public services   
32  DE        1998  GF01: General public services   
33  DE        1999  GF01: General public services   

                                     na_item obs_value  
29  TE: Total general government expenditure       7.4  
30  TE: Total general government expenditure       7.3  
31  TE: Total general government expenditure       7.3  
32  TE: Total general government expenditure       7.2  
33  TE: Total general government expenditure       6.9  

=== df_tax filtrado ===
Shape: (305, 5)
   geo time_period                         structure_name na_item obs_value
9   IT        2015  Main national accounts tax aggregates   D51C1       0.2
10  IT        2016  Main national accounts tax aggregates   D51C1       0.2
11  IT        2017  Main natio

In [14]:
# -------------------------------------------------
# 1) Pivot de gasto (df_exp): un campo por cada categoría COFOG
# -------------------------------------------------
df_exp_pivot = (
    df_exp
      .pivot_table(
          index=['geo','time_period'],
          columns='cofog',
          values='obs_value',
          aggfunc='sum'
      )
      .reset_index()
)
# Elimina el nombre de índice de columnas
df_exp_pivot.columns.name = None

# -------------------------------------------------
# 2) Pivot de ingresos (df_tax): un campo por cada código na_item
# -------------------------------------------------
df_tax_pivot = (
    df_tax
      .pivot_table(
          index=['geo','time_period'],
          columns='na_item',
          values='obs_value',
          aggfunc='sum'
      )
      .reset_index()
)
df_tax_pivot.columns.name = None

# -------------------------------------------------
# 3) Merge de gastos e ingresos
# -------------------------------------------------
df_merged = pd.merge(
    df_exp_pivot,
    df_tax_pivot,
    on=['geo','time_period'],
    how='outer'
).fillna(0)

# -------------------------------------------------
# 4) Inspección rápida
# -------------------------------------------------
print("Merged shape:", df_merged.shape)
print(df_merged.head(5))


Merged shape: (146, 14)
  geo time_period GF01: General public services GF07: Health GF09: Education  \
0  DE        1995                           7.4          6.1             4.1   
1  DE        1996                           7.3          6.2             4.2   
2  DE        1997                           7.3          6.1             4.2   
3  DE        1998                           7.2          6.1             4.1   
4  DE        1999                           6.9          6.1             4.1   

  GF1002: Old age D51C1 D51C2 D51C3 D51D D51E D59 D61 D91  
0               0     0     0     0    0    0   0   0   0  
1               0     0     0     0    0    0   0   0   0  
2               0     0     0     0    0    0   0   0   0  
3               0     0     0     0    0    0   0   0   0  
4               0     0     0     0    0    0   0   0   0  


In [15]:
import re
def normalize_cols(cols):
    new = []
    for c in cols:
        c = c.strip().lower()
        c = re.sub(r'[\s:/().-]+', '_', c)
        c = re.sub(r'_+', '_', c)
        c = c.strip('_')
        new.append(c)
    return new

df_merged.columns = normalize_cols(df_merged.columns)

# -------------------------------------------------
# 2) Vista rápida de columnas resultantes
# -------------------------------------------------
print("Columnas normalizadas:", df_merged.columns.tolist())

Columnas normalizadas: ['geo', 'time_period', 'gf01_general_public_services', 'gf07_health', 'gf09_education', 'gf1002_old_age', 'd51c1', 'd51c2', 'd51c3', 'd51d', 'd51e', 'd59', 'd61', 'd91']


In [16]:
numeric_cols = [c for c in df_merged.columns if c not in ('geo', 'time_period')]
df_merged[numeric_cols] = df_merged[numeric_cols].astype(float)

In [17]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [18]:
coll = db['EUROSTAT_SPENDING_FLOWS']
coll.drop()
coll.insert_many(df_merged.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_merged))

Registros en la colección:  146
Número de filas de df_raw:  146
